<a href="https://colab.research.google.com/github/MittalMonika/DataScience/blob/master/yolo_Sign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SignaTech: Revolutionizing Document Processing with Advanced OCR and AI-Driven Signature Detection**

Below is brief outline of the steps that need to be executed for the workflow

1. Read document (pdf) from a given location.
2. Split the document into pages and covert each page into png/jpg format and save with appropriate naming convention
3. Use  trained&deployee YOLO model to mark table boundaries
4. Extract sub Images by cutting those tables out of original images.
5. Perform OCR on tables and extract the text matrix
6. Send the text to openai and generate a summary of the table.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/yoloSign
import os

/content/drive/MyDrive/yoloSign


# 1. Read document (pdf) from a given location

To read a document on goole colab
 that isIt is easy to upload document to google drive but To read a document that is not on Google Drive and we don't want to upload it manually each time we can use Google Colab following steps


*   Upload Directly to google drive and note the path where the document is uploaded : go to the folder and upload the document "/content/drive/MyDrive/yoloSign/Pdf_document/*.pdf"
*   If is not on Google Drive and we don't want to upload it manually each time and its availabe as a online URL document (see below)


# 2. Split the document into pages and covert each page into png/jpg format and save them with appropriate naming convention

*   For this we use package pdf2image and Poppler

In [ ]:
!pip install requests PyPDF2
!pip install pytesseract
!sudo apt install tesseract-ocr


!pip install pdf2image
# Install Poppler
!apt-get install poppler-utils

In [ ]:
form_name =['f114a','f3115','f1120-RIC','f1120-REIT','f1120-PC','fw4','f1120','f8453']


In [ ]:
import os
import requests
from pdf2image import convert_from_path


form_names = [ 'fw4', 'f3115', 'f1120ric', 'f1120pc',  'f1120', 'f8453']
constant_path = '/content/drive/MyDrive/yoloSign/Pdf_document/'
output_dir = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/'

# Ensure the directory exists
os.makedirs(constant_path, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
for form_name in form_names:
    pdf_url = f"https://www.irs.gov/pub/irs-pdf/{form_name}.pdf"
    filename = f"{form_name}.pdf"
    file_path = os.path.join(constant_path, filename)

    if os.path.exists(file_path):
        print(f"The file {filename} already exists at {file_path}. No download needed.")
    else:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            with open(file_path, 'wb') as pdf_file:
                pdf_file.write(response.content)
            print(f"PDF successfully saved to {file_path}")
        else:
            print(f"Failed to retrieve the PDF document for {form_name}.")


    # Convert PDF to a list of PNG images
    images = convert_from_path(file_path)

    # Save PNG images to files
    for i, image in enumerate(images):
        image_output_path = os.path.join(output_dir, f"{form_name}_page_{i + 1}.png")
        image.save(image_output_path, "PNG")

    print(f"The images for {form_name} are saved in {output_dir}")

In [15]:
#document_name = "AutomateDocument.pdf"
#constant_path = "/content/drive/MyDrive/yolov7/Table-Extraction-PDF-2/"
#constant_path = "/content/drive/MyDrive/yoloSign/Pdf_document/"
#pdf_file = constant_path + document_name
#outdir = filename.replace(".pdf", "")
#output_dir =constant_path + "Pdf_To_Images/" + outdir
output_dir_ = output_dir +"/"
inf_dir = output_dir_ + "detect/"


specialvar = outdir+"_Employee"

# the directory from which the weights are taken for inference
#model_dir = "/content/drive/MyDrive/yolov8/runs/detect/train4/weights/best.pt"
model_dir = "/content/drive/MyDrive/yolov8/runs/detect/yolo8_best.pt"
train_data ="/content/drive/MyDrive/yolov8/Table-Extraction-PDF-2"

In [10]:
!mv  /content/drive/MyDrive/yoloSign/Pdf_document/Pdf_Images /content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images

In [5]:
fpath = "/content/drive/MyDrive/yoloSign/"
tpath = fpath + "train"
ipath =  tpath + "/images/"
lpath  = tpath + "/labels/"
bpath  =  tpath + "/box_images/"
testpath = fpath + "test"
itestpath = testpath + "/images/"
vpath = fpath + "valid"
ivpath = vpath + "/images/"
lvpath = vpath + "/labels/"

os.makedirs(tpath, exist_ok=True)
os.makedirs(ipath, exist_ok=True)
os.makedirs(lpath, exist_ok=True)
os.makedirs(bpath, exist_ok=True)
os.makedirs(testpath, exist_ok=True)
os.makedirs(itestpath, exist_ok=True)
os.makedirs(vpath, exist_ok=True)
os.makedirs(ivpath, exist_ok=True)
os.makedirs(lvpath, exist_ok=True)


# Annotation or Labelling Images

Labeling images for any YOLO version, requires a tool that can annotate images with bounding boxes and associate each box with a class label. There are several tools available for this purpose, each with its own set of features and user interfaces. The best tool can depend on the specific needs, such as ease of use, functionality, and compatibility with YOLO formats. There are some tools for image labeling like LabelImg, Labelbox, roboflow  and so on.

Roboflow has been used for the annotation of the images for YOLO, the platform typically manages the conversion of annotation coordinates to the format required by YOLO models. YOLO uses a specific bounding box format which includes:

The center of the bounding box (x, y) relative to the width and height of the image.
The width and height of the bounding box relative to the width and height of the image.

These values are normalized to be between 0 and 1. Here's the format:

<class_id> <x_center> <y_center> <width> <height>

Each line in a YOLO annotation .txt file corresponds to a bounding box in the respective image and follows the format mentioned above.

e.g.

0   0.496875 0.77578125 0.9265625 0.0828125

here 0 indicates the class . we can multiclass labelling also


# Generating Simulated dataset

To achieve the goal of creating a generated\simulated dataset for YOLO training, we'll need to follow a process that involves cutting out the annotated portion of the image and then pasting it onto various sections of other documents. Let's break down the steps

# Cutting the Annotated Portion from the Image
Based on the YOLO annotation line (e.g., 0 0.51 0.33 0.27 0.50), we need to extract the corresponding part of the image. The annotation gives us the center coordinates of the bounding box, along with its width and height relative to the image size.

1. **Calculate Absolute Bounding Box Coordinates**:
      * Convert the relative coordinates to absolute pixel values based on the image size.
      * Calculate the top-left corner of the bounding box from its center coordinates.
2. **Crop the Image**
      * Use these coordinates to crop the region from the image.

In [13]:
from PIL import Image,ImageDraw

def crop_annotation(image_path, annotation):
    # Load the image
    image = Image.open(image_path)
    img_width, img_height = image.size

    # Parse the annotation
    _, x_center, y_center, width, height = map(float, annotation.split())

    # Convert YOLO format to pixel values
    box_width = width * img_width
    box_height = height * img_height
    x_center *= img_width
    y_center *= img_height

    # Calculate top-left corner of the bounding box
    x1 = int(x_center - box_width / 2)
    y1 = int(y_center - box_height / 2)

    # Crop the image
    cropped_image = image.crop((x1, y1, x1 + int(box_width), y1 + int(box_height)))

    return cropped_image

In [16]:
label_image = "0 0.506875 0.77578125 0.9265625 0.0898125"
file_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4_page_1.png'
cropped_image = crop_annotation(file_path,label_image)
cropped_image.save(bpath+'/cropped_image_'+ specialvar +'.png')


# Creating a Generated Dataset
Paste this cropped section onto various parts of other documents. Here lets start with one document

**Prepare Target Documents:**

  * Have a set of target documents (images) where we can paste the cropped section.

**Random Placement of Cropped Section:**

  * Choose random positions on these target documents to paste the cropped section.

  * Ensure the pasted section fits within the boundaries of the target documents.

  
**Save the Modified Documents:**

  * Save each modified document as a new image file.


In [14]:
import random
def paste_on_background(background_path, cropped_image, output_path,output_box_path, position):
    background = Image.open(background_path)
    background_width, background_height = background.size

    # Create a copy of the background for drawing the bounding box
    background_with_box = background.copy()


    # Paste the cropped image onto the background at the specified position
    background.paste(cropped_image, position, cropped_image if cropped_image.mode == 'RGBA' else None)
    background_with_box.paste(cropped_image, position, cropped_image if cropped_image.mode == 'RGBA' else None)


    # Draw bounding box for visualization (optional)
    draw = ImageDraw.Draw(background_with_box)
    x1, y1 = position
    x2, y2 = x1 + cropped_image.width, y1 + cropped_image.height
    draw.rectangle([x1, y1, x2, y2], outline="red")

    # Save the result
    background.save(output_image_path)
    background_with_box.save(output_box_path.replace('.png', '_bbox.png'))
    print(f"Saved images to {output_image_path} and {output_box_path.replace('.png', '_bbox.png')}")


    # Calculate YOLO format coordinates
    bbox_x_center = (x1 + x2) / 2 / background_width
    bbox_y_center = (y1 + y2) / 2 / background_height
    bbox_width = cropped_image.width / background_width
    bbox_height = cropped_image.height / background_height

    yolo_format = (bbox_x_center, bbox_y_center, bbox_width, bbox_height)
    return yolo_format

In [12]:
output_dir = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/'
output_dir_ = output_dir +"/"
inf_dir = output_dir_ + "detect/"
os.makedirs(output_dir_, exist_ok=True)
os.makedirs(inf_dir, exist_ok=True)

# Preparing Train dataset

1.  Took 3rd page of all the document and pasted the cropped image of the first page of sign image of fw4 placed at randomly generated various positon



In [ ]:
label_image = "0 0.506875 0.77578125 0.9265625 0.0898125"
file_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4_page_1.png'
cropped_image = crop_annotation(file_path,label_image)
cropped_image.save(bpath+'/cropped_image_'+ specialvar +'.png')

specialvar = form_names[0]
exclusion_pattern = specialvar + "_page_1.png"
document_paths = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('_page_3.png') and  f != exclusion_pattern ]
for doc_path in document_paths:
  for i in range(0, 20):
    pos_y = random.randint(40, 2000)
    position = (100, pos_y)
    output_image_path = os.path.join(ipath, os.path.basename(doc_path).replace('.png','') + '_'+specialvar + '_' +str(i) +'.png' )
    output_box_path = os.path.join(bpath, os.path.basename(doc_path).replace('.png','') + '_'+specialvar + '_' +str(i) +'_bbox.png' )

    yolo_coordinates = paste_on_background(doc_path, cropped_image, output_image_path, output_box_path, position)

    with open(os.path.join(lpath, os.path.basename(doc_path).replace('.png','') + '_'+specialvar + '_' +str(i) + '.txt'), 'w') as file:
        file.write(f"0 {yolo_coordinates[0]} {yolo_coordinates[1]} {yolo_coordinates[2]} {yolo_coordinates[3]}")


2. Copy all other images (not the 1st) as they dnt have sign document with a empty label file for training dataset.

In [33]:
document_paths = [os.path.join(output_dir, f) for f in os.listdir(output_dir) ]
from shutil import copyfile
for doc_path in document_paths:
    # Extract the base file name without page number and extension
    base_file_name = os.path.basename(doc_path).rsplit('_', 2)[0]

    page_num = 2  # Starting from the second page
    while True:
        # Construct the new page path
        page_path = os.path.join(os.path.dirname(doc_path), f"{base_file_name}_page_{page_num}.png")
        if not os.path.exists(page_path):
            break  # Stop if the page does not exist

        # Copy image to training images directory
        dest_image_path = os.path.join(ipath, f"{base_file_name}_page_{page_num}.png")
        copyfile(page_path, dest_image_path)

        # Create an empty file in labels directory
        with open(os.path.join(lpath, f"{base_file_name}_page_{page_num}.txt"), 'w') as file:
            pass  # Empty file

        page_num += 1

# Preparing data for Validation dataset

Coping the 20% of generated train data to validation dataset

In [18]:
import os
import shutil
import random



# Get a list of filenames (without extensions) in the training images directory
train_filenames = [os.path.splitext(f)[0] for f in os.listdir(ipath) if f.endswith('.png')]

# Decide how many images you want to move to validation (e.g., 20% of the training set)
num_valid = int(len(train_filenames) * 0.2)

# Randomly select images for validation
valid_filenames = random.sample(train_filenames, num_valid)

# Copy selected images and their corresponding label files to validation directories
for filename in valid_filenames:
    # Copy image
    src_image_path = os.path.join(ipath, filename + '.png')
    dst_image_path = os.path.join(ivpath, filename + '.png')
    shutil.copyfile(src_image_path, dst_image_path)

    # Copy label file
    src_label_path = os.path.join(lpath, filename + '.txt')
    dst_label_path = os.path.join(lvpath, filename + '.txt')
    shutil.copyfile(src_label_path, dst_label_path)

print(f"Copied {num_valid} images and their labels to the validation directories.")


Copied 27 images and their labels to the validation directories.


# Preparing data for Test dataset.

Just coping all the first page of all the form for the test data

In [ ]:
import os
from shutil import copyfile


# Copy all files ending with 'page_1.png'
for filename in os.listdir(output_dir):
    if filename.endswith('fw4_page_1.png'):
        source_file_path = os.path.join(output_dir, filename)
        destination_file_path = os.path.join(itestpath, filename)
        copyfile(source_file_path, destination_file_path)
        print(f"Copied {filename} to {itestpath}")

## Training YOLOv8 on a Custom Dataset (identify Sign Boxes)

This notebook is based on the ultralytics package and performs training on own custom objects and here i am using a document set and trying to tag sign boxesfrom the scanned/screenshot images


### **Steps that are followed**

To train the yolov8 follow steps:

* Install YOLOv8 and all its dependencies
* Load custom dataset here i took document-parts dataset
* Run YOLOv8 training
* Evaluate YOLOv8 performance
* Run YOLOv8 inference on test images

In [ ]:
#recommended way of installing through pip
!pip install ultralytics

# Train YOLOv8 on a custom dataset

In [21]:
model_dir_train = "/content/drive/MyDrive/yolov8/runs/detect/yolo8_best.pt"
train_data = "/content/drive/MyDrive/yoloSign/"


In [23]:
!yolo task=detect \
mode=train \
model={model_dir_train} \
data={train_data}/data.yaml \
epochs=15 \
imgsz=640

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/yolov8/runs/detect/yolo8_best.pt, data=/content/drive/MyDrive/yoloSign//data.yaml, epochs=15, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

# Validate with a new model

When the *training* is over, we can validate the new model on images it has not seen before. Therefore, when creating a dataset, we divide it into three parts, and one of them that we will use now as a test dataset.

In [26]:
model_dir = "runs/detect/train/weights/best.pt"
testpath = "/content/drive/MyDrive/yoloSign/test/images/"

In [27]:
!yolo task=detect \
mode=val \
model= {model_dir} \
data= {train_data}/data.yaml

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/yoloSign/valid/labels.cache... 27 images, 7 backgrounds, 0 corrupt: 100% 28/28 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:03<00:00,  1.67s/it]
                   all         28         21      0.952          1      0.986      0.944
Speed: 3.2ms preprocess, 17.6ms inference, 0.0ms loss, 40.2ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


# Testing on a new data

In [28]:
!yolo task=detect \
mode=predict \
model={model_dir} \
conf=0.25 \
source={testpath} \
name={inf_dir} \
save_txt= True

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/drive/MyDrive/yoloSign/test/images/fw4_page_1.png: 640x512 1 sign, 83.0ms
Speed: 5.6ms preprocess, 83.0ms inference, 534.0ms postprocess per image at shape (1, 3, 640, 512)
Results saved to /content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/detect
1 label saved to /content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/detect/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict


# Backup

In [ ]:





document_paths = [os.path.join(output_dir, f) for f in os.listdir(output_dir) ]
print(document_paths)
from shutil import copyfile
for doc_path in document_paths:
    # Extract the base file name without page number and extension
    base_file_name = os.path.basename(doc_path).rsplit('_', 2)[0]

    page_num = 2  # Starting from the second page
    while True:
        # Construct the new page path
        page_path = os.path.join(os.path.dirname(doc_path), f"{base_file_name}_page_{page_num}.png")
        if not os.path.exists(page_path):
            break  # Stop if the page does not exist

        # Copy image to training images directory
        dest_image_path = os.path.join(ipath, f"{base_file_name}_page_{page_num}.png")
        copyfile(page_path, dest_image_path)

        # Create an empty file in labels directory
        with open(os.path.join(lpath, f"{base_file_name}_page_{page_num}.txt"), 'w') as file:
            pass  # Empty file

        page_num += 1

In [16]:
specialvar = form_names[0]
form_names = [ 'fw4', 'f3115', 'f1120ric', 'f1120pc',  'f1120', 'f8453']
exclusion_pattern = specialvar + "_page_1.png"
document_paths = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('_page_1.png') and  f != exclusion_pattern ]
print(document_paths)

['/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/f3115_page_1.png', '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/f1120ric_page_1.png', '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/f1120pc_page_1.png', '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/f1120_page_1.png', '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/f8453_page_1.png']


In [ ]:
from pdf2image import convert_from_path
# Specify the PDF file path
os.makedirs(output_dir, exist_ok=True)

# Convert PDF to a list of PNG images
images = convert_from_path(file_path)

# Save PNG images to files
for i, image in enumerate(images):
    image.save(os.path.join(output_dir, f"output_page_{i + 1}.png"), "PNG")
print("The images are saved in ", output_dir)

The images are saved in  /content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4


In [33]:
label_image = "0 0.506875 0.77578125 0.9265625 0.0898125"
file_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
cropped_image = crop_annotation(file_path,label_image)
cropped_image.save(bpath+'/cropped_image_'+ specialvar +'.png')



background_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_3.png'
for i in range(0, 100):
    pos_y = random.randint(40, 2000)
    position = (100, pos_y)
    output_image_path = f'{ipath}{specialvar}_{i}.png'
    output_boximage_path = f'{bpath}{specialvar}_{i}.png'
    yolo_coordinates = paste_on_background(background_path, cropped_image, output_image_path,output_boximage_path, position)

    with open(f'{lpath}{specialvar}_{i}.txt', 'w') as file:
        file.write(f"0 {yolo_coordinates[0]} {yolo_coordinates[1]} {yolo_coordinates[2]} {yolo_coordinates[3]}")

Saved images to /content/drive/MyDrive/yoloSign/train/images/fw4_Employee_0.png and /content/drive/MyDrive/yoloSign/train/box_images/fw4_Employee_0_bbox.png
Saved images to /content/drive/MyDrive/yoloSign/train/images/fw4_Employee_1.png and /content/drive/MyDrive/yoloSign/train/box_images/fw4_Employee_1_bbox.png
Saved images to /content/drive/MyDrive/yoloSign/train/images/fw4_Employee_2.png and /content/drive/MyDrive/yoloSign/train/box_images/fw4_Employee_2_bbox.png
Saved images to /content/drive/MyDrive/yoloSign/train/images/fw4_Employee_3.png and /content/drive/MyDrive/yoloSign/train/box_images/fw4_Employee_3_bbox.png
Saved images to /content/drive/MyDrive/yoloSign/train/images/fw4_Employee_4.png and /content/drive/MyDrive/yoloSign/train/box_images/fw4_Employee_4_bbox.png
Saved images to /content/drive/MyDrive/yoloSign/train/images/fw4_Employee_5.png and /content/drive/MyDrive/yoloSign/train/box_images/fw4_Employee_5_bbox.png
Saved images to /content/drive/MyDrive/yoloSign/train/imag

In [14]:
constant_path = "/content/drive/MyDrive/yoloSign/Pdf_document/"

import os
import requests
# URL of the PDF document
pdf_url = "https://www.irs.gov/pub/irs-pdf/fw4.pdf"
# Extracting the filename from the URL
filename = pdf_url.split('/')[-1]
file_path = os.path.join(constant_path, filename)
# Check if the file already exists
if os.path.exists(file_path):
    print(f"The file {filename} already exists at {file_path}. No download needed.")
else:
    # Fetch the PDF content from the URL
    response = requests.get(pdf_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Save the PDF to the specified directory
        with open(file_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        print(f"PDF successfully saved to {file_path}")
    else:
        print("Failed to retrieve the PDF document.")


The file fw4.pdf already exists at /content/drive/MyDrive/yoloSign/Pdf_document/fw4.pdf. No download needed.


In [ ]:
from PIL import Image
import cv2
background_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
overlay_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_3.png'
output_path = 'image_with_overlay.png'
background = Image.open(background_path)
overlay = Image.open(overlay_path)
# Print out their modes
print(f"Background mode: {background.mode}")
print(f"Overlay mode: {overlay.mode}")

# Convert overlay to 'RGBA' if it's not already
if overlay.mode != 'RGBA':
    overlay = overlay.convert('RGBA')


# Calculate the new height to maintain the aspect ratio
aspect_ratio = overlay.width / overlay.height
new_height = int((overlay.width*0.95) / aspect_ratio)

# Resize the overlay image to match the width of the background
overlay = overlay.resize((int(overlay.width*0.95), new_height))

position = (100, 1625)
print(f"Background size: {background.size}")
print(f"Resized overlay size: {overlay.size}")
print(f"Position to paste: {position}")




(x,y) = position

box = (x,y,x+overlay.width, y+overlay.height)
cropped_region = background.crop(box)
pos_o_x = 100
pos_o_y = 1625
background.paste(cropped_region,(pos_o_x,pos_o_y))

    # Paste the overlay image onto the background at the specified position
background.paste(overlay, position, overlay)

#background.paste(overlay, position,overlay)

# Save the combined image
output_path = 'image_with_overlay.png'
background.save(output_path)
print(f"Image saved to {output_path}")

Background mode: RGB
Overlay mode: RGB
Background size: (1700, 2200)
Resized overlay size: (1615, 2090)
Position to paste: (100, 1625)
Image saved to image_with_overlay.png


In [ ]:
#Variable Names
background_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
overlay_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
output_path = 'image_with_overlay.png'

from PIL import Image
import cv2

def get_image_dimensions(image_path):
    # Open the subimage using PIL/Pillow
    image = Image.open(image_path)

    # Get the width and height of the subimage
    width, height = image.size

    return width, height
def overlay_subimage(background_path, overlay_path, output_path, position):
    # Open the background and overlay images
    background = Image.open(background_path)
    overlay = Image.open(overlay_path)

    # Resize the overlay image to make sure it fits within the background
    max_width = background.width - position[0]  # Maximum width the overlay can have
    aspect_ratio = overlay.width / overlay.height
    new_height = int(min(overlay.height, (max_width / aspect_ratio)))
    new_width = int(min(overlay.width, max_width))
    overlay_resized = overlay.resize((new_width, new_height))

    # Calculate the box for cropping (ensure it's within the background boundaries)
    x, y = position
    box_width = min(overlay_resized.width, background.width - x)
    box_height = min(overlay_resized.height, background.height - y)
    box = (x, y, x + box_width, y + box_height)

    # Paste the resized overlay image onto the background at the specified position
    background.paste(overlay_resized, position, overlay_resized if overlay_resized.mode == 'RGBA' else None)

    # Save the result
    background.save(output_path)
    print(f"Saved image to {output_path}")




    # Calculate bounding box parameters
    (x,y) = position
    height, width = background.height, background.width
    center_x = (x + overlay.width / 2) / width
    center_y = (y + overlay.height / 2) / height
    normalized_width = overlay.width / width
    normalized_height = overlay.height / height

    # Express bounding box in YOLO format
    yolo_format = (center_x, center_y, normalized_width, normalized_height)

    # Save the result

    return yolo_format


def draw_bounding_box(img_path,ybox):
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    center_x, center_y, normalized_width, normalized_height = ybox

    # Draw bounding box on the image for visualization
    box_x = int((center_x - normalized_width / 2) * width)
    box_y = int((center_y - normalized_height / 2) * height)
    cv2.rectangle(img, (box_x, box_y), (box_x + int(normalized_width * width), box_y + int(normalized_height * height)), (0, 255, 0), 2)

    return img





# Example usage
subimage_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
bkg_width, bkg_height = get_image_dimensions(background_path)

print("Subimage dimensions:", bkg_width, "x", bkg_height)

import random

pos_o_x = 100
pos_o_y = 1625

for i in range(0,11):
    pos_y = random.randint(40,2000)
    print(pos_y)
    position = (100, pos_y)  # Example coordinates
    ybox = overlay_subimage(background_path, overlay_path, output_path, position)
    # Display the image with bounding box
    img = draw_bounding_box(output_path, ybox)
    cv2.imwrite('output_image_?i?.jpg'.replace('?i?',str(i)), img)
    print(i,pos_y)

#cv2.imshow('Image with bounding box', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

Subimage dimensions: 1700 x 2200
1123
100 1123 1715 3213
0 1123
1817
100 1817 1715 3907
1 1817
746
100 746 1715 2836
2 746
659
100 659 1715 2749
3 659
1550
100 1550 1715 3640
4 1550
1129
100 1129 1715 3219
5 1129
1608
100 1608 1715 3698
6 1608
1184
100 1184 1715 3274
7 1184
1244
100 1244 1715 3334
8 1244
765
100 765 1715 2855
9 765
1544
100 1544 1715 3634
10 1544


In [ ]:
from PIL import Image
background_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
overlay_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
output_path = 'image_with_overlay.png'

# Load the background and overlay images to get their dimensions
background = Image.open(background_path)
overlay = Image.open(overlay_path)
bkg_width, bkg_height = background.size

# Resize overlay and get new dimensions
aspect_ratio = overlay.width / overlay.height
max_width = bkg_width - 100  # Assuming '100' is the fixed x-coordinate for the overlay
new_height = int((overlay.width * 0.95) / aspect_ratio)
new_width = int(min(overlay.width * 0.95, max_width))

# Adjust new_height if it exceeds background boundaries
if new_height > (bkg_height - 100):  # Assuming '100' is the minimum y-coordinate
    new_height = bkg_height - 100
    new_width = int(new_height * aspect_ratio)


def overlay_subimage(background_path, overlay_path, output_path, position, new_width, new_height):
    # Open the background and overlay images
    background = Image.open(background_path)
    overlay = Image.open(overlay_path)

    # Resize the overlay
    overlay_resized = overlay.resize((new_width, new_height))

    # Paste the resized overlay image onto the background
    background.paste(overlay_resized, position)


    # Calculate bounding box parameters
    (x,y) = position
    height, width = background.height, background.width
    center_x = (x + overlay.width / 2) / width
    center_y = (y + overlay.height / 2) / height
    normalized_width = overlay.width / width
    normalized_height = overlay.height / height

    # Express bounding box in YOLO format
    yolo_format = (center_x, center_y, normalized_width, normalized_height)

    # Save the result
    background.save(output_path)
    return yolo_format


def draw_bounding_box(img_path,ybox):
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    center_x, center_y, normalized_width, normalized_height = ybox

    # Draw bounding box on the image for visualization
    box_x = int((center_x - normalized_width / 2) * width)
    box_y = int((center_y - normalized_height / 2) * height)
    cv2.rectangle(img, (box_x, box_y), (box_x + int(normalized_width * width), box_y + int(normalized_height * height)), (0, 255, 0), 2)

    return img

import random

for i in range(0, 11):
    # Ensure the overlay is always within the background boundaries
    pos_y = random.randint(40, bkg_height - new_height)
    position = (100, pos_y)  # Example coordinates

    # Call the function to overlay the image
    ybox = overlay_subimage(background_path, overlay_path, f'output_image_{i}.png', position,new_width, new_height)

    # Display the image with bounding box
    img = draw_bounding_box(output_path, ybox)
    cv2.imwrite('output_image_?i?.jpg'.replace('?i?',str(i)), img)
    print(i,pos_y)



0 43
1 73
2 70
3 108
4 43
5 82
6 94
7 40
8 50
9 67
10 49


In [ ]:
#Variable Names
background_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
overlay_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_3.png'
output_path = 'image_with_overlay.png'

from PIL import Image
import cv2

def get_image_dimensions(image_path):
    # Open the subimage using PIL/Pillow
    image = Image.open(image_path)

    # Get the width and height of the subimage
    width, height = image.size

    return width, height
def overlay_subimage(background_path, overlay_path, output_path, position):
    # Open the background and overlay images
    background = Image.open(background_path)
    overlay = Image.open(overlay_path)

    # Resize the overlay image if necessary
    aspect_ratio = overlay.width / overlay.height
    new_height = int((overlay.width * 0.95) / aspect_ratio)
    overlay_resized = overlay.resize((int(overlay.width * 0.95), new_height))

    # Paste the resized overlay image onto the background at the specified position
    background.paste(overlay_resized, position, overlay_resized if overlay_resized.mode == 'RGBA' else None)



    # Calculate bounding box parameters
    (x,y) = position
    height, width = background.height, background.width
    center_x = (x + overlay.width / 2) / width
    center_y = (y + overlay.height / 2) / height
    normalized_width = overlay.width / width
    normalized_height = overlay.height / height

    # Express bounding box in YOLO format
    yolo_format = (center_x, center_y, normalized_width, normalized_height)

    # Save the result
    background.save(output_path)
    return yolo_format


def draw_bounding_box(img_path,ybox):
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    center_x, center_y, normalized_width, normalized_height = ybox

    # Draw bounding box on the image for visualization
    box_x = int((center_x - normalized_width / 2) * width)
    box_y = int((center_y - normalized_height / 2) * height)
    cv2.rectangle(img, (box_x, box_y), (box_x + int(normalized_width * width), box_y + int(normalized_height * height)), (0, 255, 0), 2)

    return img





# Example usage
subimage_path = '/content/drive/MyDrive/yoloSign/Pdf_document/Pdf_To_Images/fw4/output_page_1.png'
bkg_width, bkg_height = get_image_dimensions(background_path)

print("Subimage dimensions:", bkg_width, "x", bkg_height)

import random

pos_o_x = 100
pos_o_y = 1625

for i in range(0,11):
    pos_y = random.randint(40,2000)
    print(pos_y)
    position = (100, pos_y)  # Example coordinates
    ybox = overlay_subimage(background_path, overlay_path, output_path, position)
    # Display the image with bounding box
    img = draw_bounding_box(output_path, ybox)
    cv2.imwrite('output_image_?i?.jpg'.replace('?i?',str(i)), img)
    print(i,pos_y)

#cv2.imshow('Image with bounding box', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

Subimage dimensions: 1700 x 2200
1476
0 1476
1628
1 1628
1908
2 1908
326
3 326
674
4 674
892
5 892
1359
6 1359
1686
7 1686
1686
8 1686
1443
9 1443
469
10 469
